<a href="https://colab.research.google.com/github/torresmateo/penguin-tf-workshop/blob/master/D1_E1_Fashion_MNIST_SOLUCION.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Fashion MNIST - Clasificación de Imágenes
Los siguientes ejercicios son para familiarizarse con una red neuronal simple. Se implementará un modelo similar a MNIST, cuya única diferencia son los *labels*. 

In [ ]:
# Se incluyen las bibliotecas necesarias
%tensorflow_version 2.x
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Ejercicio 1 - Descargar el *dataset* 

Leer la documentación de [`fashion_mnist`](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/fashion_mnist) y cargar el dataset en memoria en variables de training y testing.

Luego, crear una lista con los nombres de las clases del *dataset*. Los nombres de las clases (o *labels*) pueden encontrarse en el repositorio [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) (leer la documentación)

In [ ]:
(train_imgs, train_labels), (test_imgs, test_labels) = tf.keras.datasets.fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Ejercicio 2 - Explorar el *dataset*

keras ya particiona el *dataset* en *training* y *testing*, pero siempre es bueno entender el formato de los datos. 

Explora el *dataset* para obtener las respuestas a las siguientes preguntas:
1. ¿Cuántas imágenes de *training* hay en el *dataset*?
2. ¿Cuántas imágenes de *testing*?
3. ¿Cuál es la resolución de las imágenes en el dataset?
4. ¿Cuál es el tipo de dato cada pixel? ¿Y de los *labels*?

In [ ]:
print('ejemplos de training:', train_imgs.shape[0])
print('ejemplos de testing', test_imgs.shape[0])
print('resolución de las imágenes', train_imgs[0].shape)
print('tipo de dato de los píxeles', type(train_imgs[0,0,0]))
print('tipo de dato de los labels', type(train_labels[0]))

# Función de ayuda - Visualizar el *dataset*

Se provee una función que permite visualizar ejemplos individuales en el *dataset*. 


In [ ]:
def ver_ejemplo(training_dataset, training_labels, indice, clases):
    """
    Visualizar una imágen del dataset

    Parameters
    ----------
    training_dataset : np.ndarray
        Un dataset de imágenes
    training_labels : np.ndarray
        Labels correspondientes al `training_dataset`
    indice : int
        el índice al ejemplo que se quiere visualizar
    clases : list
        La lista de clases que se creó en el Ejercicio 1
    """
    fig, ax = plt.subplots(figsize=(2,2), dpi=100)
    ax.set(title=clases[training_labels[indice]])
    im = ax.imshow(training_dataset[indice], cmap='gray')
    fig.colorbar(im, ax=ax)
    ax.grid(False)
    plt.show()

In [ ]:
ver_ejemplo(train_imgs, train_labels, 0, class_names)

#Ejercicio 3 - normalizar el *dataset*

Como fue explicado en las diapositivas, muchas funciones de activación que se usan en las redes neuronales funcionan mejor cuando el **rango** de las variables está entre `0` y `1`. En este ejercicio, normaliza las **imágenes** de *training* y *testing* de tal manera a que estén en el rango correcto.

In [ ]:
train_imgs = train_imgs / np.max(train_imgs)
test_imgs = test_imgs / np.max(test_imgs)

# Ejercicio 4 - Explorar la red neuronal

Probablemente hayas notado que el formato de Fashion MNIST es prácticamente idéntico al de MNIST (imágenes de 28x28 píxeles en escala de grises, y 10 clases). Por lo tanto, utilizaremos el mismo modelo y se discutirán algunos cambios que pueden mejorar la calidad de nuestro modelo.

Completa las variables en el modelo para utilizar el dataset que cargaste en el
Ejercicio 1

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(28, 28)), 
                                    tf.keras.layers.Dense(100, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_imgs,train_labels, epochs=10)

In [ ]:
loss, acc = model.evaluate(test_imgs, test_labels)

Hacemos las predicciones para todo el *testing set* para poder analizar los resultados 

In [ ]:
predicciones = model.predict(test_imgs)

# Función de ayuda - Visualizar las predicciones

Se provee una función que permite visualizar ejemplos individuales en el *dataset* y la predicción del modelo en paralelo. 

In [ ]:
def ver_prediccion(test_dataset,test_labels, predicciones, indice, clases):
    """
    Visualizar una imágen del dataset

    Parameters
    ----------
    test_dataset : np.ndarray
        Un dataset de imágenes
    test_labels : np.ndarray
        Labels correspondientes al `test_dataset`
    predicciones : np.ndarray
        array con predicciones del modelo
    indice : int
        el índice al ejemplo que se quiere visualizar
    clases : list
        La lista de clases que se creó en el Ejercicio 1
    """
    fig, ax = plt.subplots(1,2, dpi=100)
    ax[0].set(title=clases[test_labels[indice]])
    im = ax[0].imshow(test_dataset[indice], cmap='gray')
    #fig.colorbar(im, ax=ax[0])
    ax[0].grid(False)
    ax[0].get_xaxis().set_ticks([])
    ax[0].get_yaxis().set_ticks([])
    ax[1].set_xticklabels(clases, rotation=90)
    ax[1].get_yaxis().set_ticks([])
    bars = ax[1].bar(clases, predicciones[indice], color='gray')
    prediccion = np.argmax(predicciones[indice])
    bars[prediccion].set_color('red')
    bars[test_labels[indice]].set_color('blue')
    plt.show()

In [ ]:
ver_prediccion(test_imgs, test_labels, predicciones, 9, class_names)

# Ejercicio 5 - Analizar los errores del modelo

El modelo simple funciona bastante bien, pero hay algunos errores. Identifica los ejemplos que fueron mal clasificados por el modelo, y visualiza 5 de ellos.

In [ ]:
preds = np.argmax(predicciones, axis=1)
errores = np.where(preds != test_labels)[0]

for e in errores[:5]:
    ver_prediccion(test_imgs, test_labels, predicciones, e, class_names)